# Script to Consolidate Data and Create a Day Count

In [1]:
import pandas as pd
import glob

In [2]:
path = "D:/Dropbox/Data/R Shiny/20-03-20 Covid-19"

## Combine all the files together

In [3]:
all_files = glob.glob(path + "/csse_covid_19_daily_reports/*.csv")

li = []

# for files in folder, create a dataframe out of them
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    # create a date field that is generated from the file name
    df['date'] = filename.split('\\')[-1].split('.')[0]
    # add data to list
    li.append(df)

# concatinate all data (union) into final dataframe    
frame = pd.concat(li, axis=0, ignore_index=True)

# Change Country Names for consistency
frame["Country/Region"].replace({"occupied Palestinian territory": "Palestine", "Mainland China": "China", "Taiwan*": "Taiwan", "UK": "United Kingdom", "The Bahamas": "Bahamas, The", "Iran (Islamic Republic of)": "Iran", "Hong Kong SAR": "Hong Kong", "The Gambia": "Gambia, The", "Viet Nam": "Vietnam"}, inplace=True)

In [4]:
# aggregate numbers by country
country_df = frame.groupby(['Country/Region','date']).sum().reset_index()

## Aggregate the numbers on a country level

In [5]:
country_list = []

# split the dataframe by country
for country, country_df in country_df.groupby('Country/Region'):
    # Add a day field and number it chronologically
    country_df = country_df.reset_index()
    country_df['Days'] = country_df.index + 1
    
    # Get deltas for deaths, recovered, and then new cases using pd.diff() function
    country_df['New_Deaths'] = country_df['Deaths'].diff()
    country_df['New_Recovered'] = country_df['Recovered'].diff()
    country_df['New_Confirmed'] = country_df['Confirmed'].diff()
    
    
    # add data to list
    country_list.append(country_df)
    
# concatinate all data (union) into final dataframe    
frame2 = pd.concat(country_list, axis=0, ignore_index=True)

## Cleaning the data

In [6]:
# delete newly created fields
del frame2['index']

In [7]:
# rearrange fields for easier reading
final_table = frame2[['date','Days','Country/Region','Confirmed','New_Confirmed','Deaths','New_Deaths','Recovered','New_Recovered','Latitude','Longitude']]
#replace NaN with empty string
final_table = final_table.fillna(0)
# Get active cases for the day
final_table['Active_Cases'] = final_table['Confirmed'] - final_table['Deaths'] - final_table['Recovered']

In [8]:
# for row in dataframe
for i, row in final_table.iterrows():
    # if Days == 1
    if (row['Days'] == 1):
        # C_delta == Confirmed, D_delta == Deaths, R_delta == Recovered
        final_table.at[i,'New_Confirmed'] = row['Confirmed']
        final_table.at[i,'New_Deaths'] = row['Deaths']
        final_table.at[i,'New_Recovered'] = row['Recovered']

In [9]:
final_table[final_table['Country/Region'] == 'United Kingdom']

,date,Days,Country/Region,Confirmed,New_Confirmed,Deaths,New_Deaths,Recovered,New_Recovered,Latitude,Longitude,Active_Cases
3756,01-31-2020,1,United Kingdom,2.0,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.0
3757,02-01-2020,2,United Kingdom,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.0
3758,02-02-2020,3,United Kingdom,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.0
3759,02-03-2020,4,United Kingdom,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.0
3760,02-04-2020,5,United Kingdom,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.0
3761,02-05-2020,6,United Kingdom,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.0
3762,02-06-2020,7,United Kingdom,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.0
3763,02-07-2020,8,United Kingdom,3.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,3.0
3764,02-08-2020,9,United Kingdom,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,3.0
3765,02-09-2020,10,United Kingdom,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,3.0


In [10]:
final_table.to_csv(path + "/transformed.csv", index=False)